<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек,-чтение-и-изучение-файла" data-toc-modified-id="Импорт-библиотек,-чтение-и-изучение-файла-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек, чтение и изучение файла</a></span></li><li><span><a href="#Разделение-датасета-на-выборки" data-toc-modified-id="Разделение-датасета-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение датасета на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-моделей-на-тестовой-выборке" data-toc-modified-id="Проверка-моделей-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка моделей на тестовой выборке</a></span></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».  

Нужно подобрать модель для задачи классификации, которая порекомендует пользователю один из предложенных тарифов.
Исходные данные - описание поведения пользователей.  

Источник данных: Яндекс. Практикум

## Импорт библиотек, чтение и изучение файла

In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.dummy import DummyClassifier

In [27]:
users_behavior = pd.read_csv('/datasets/users_behavior.csv')

In [28]:
users_behavior.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Описание данных  

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:  

`сalls` — количество звонков,  
`minutes` — суммарная длительность звонков в минутах,  
`messages` — количество sms-сообщений,  
`mb_used` — израсходованный интернет-трафик в Мб,  
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

In [29]:
users_behavior.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [30]:
users_behavior['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [31]:
users_behavior.duplicated().sum()

0

In [32]:
users_behavior.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [33]:
users_behavior.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


**Вывод**
***
В датасете 3214 строк, из них:
- тариф смарт(0) - 2229  
- тариф ультра(1) - 985  

Дубликатов, пропущенных значений не обнаружено.  
Минимальное значение звонков/смс/интернета - 0.

## Разделение датасета на выборки

Модель должна порекомендовать на какой тариф лучше перейти пользователю, значит целевой признак это тариф - is_ultra. 

Целевой признак категориальный (два тарифа 0 и 1), поэтому решается задача классификации.  
Разделим датасет на целевой признак и на признаки для обучения моделей.

In [34]:
features_users_behavior = users_behavior.drop('is_ultra', axis=1)
target_users_behavior = users_behavior['is_ultra']

In [35]:
print('Размер датасета с признаками', features_users_behavior.shape)
print('Размер датасета с целевым признаком', target_users_behavior.shape)

Размер датасета с признаками (3214, 4)
Размер датасета с целевым признаком (3214,)


Разделим данные на три части в два этапа: обучающую, валидационную и тестовую выборки в пропорции 3:1:1.  
Соответственно, обучающая выборка - 60% от всех данных, 20% для валидационной и тестовой выборок.

In [36]:
# разделяем первым этапом тренировочную выборку - 60% и тестовую - 40%
features_train, features_valid, target_train, target_valid = train_test_split(features_users_behavior,
                                                                             target_users_behavior,
                                                                             test_size=0.4, 
                                                                             random_state=12345)

# разделяем тестовую - 40% в равных долях на две - валидационную и тестовую выборки
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                           target_valid, 
                                                                           test_size=0.5,
                                                                           random_state=12345)

In [37]:
print('Размер тренировочной выборки признаков:', features_train.shape, 'целевого признака:', target_train.shape, sep='\n')
print('Размер валидационной выборки признаков:', features_valid.shape, 'целевого признака:', target_valid.shape, sep='\n')
print('Размер тестовой выборки признаков:', features_test.shape, 'целевого признака:', target_test.shape, sep='\n')

Размер тренировочной выборки признаков:
(1928, 4)
целевого признака:
(1928,)
Размер валидационной выборки признаков:
(643, 4)
целевого признака:
(643,)
Размер тестовой выборки признаков:
(643, 4)
целевого признака:
(643,)


**Вывод**
***
Исходная выборка разделена на три:
- 60% обучающая выборка.
- по 20% валидационная и тестовая выборки.

## Исследование моделей

### Дерево решений 

Обучим модель дерева решений на обучающей выборке features_train, target_train.   
Переберем вложенным циклом гиперпараметры (max_depth, min_samples_split), и найдем модель с гиперпараметрами, которая дает наилучший результат точности предсказаний (accuracy) на валидационной выборке и сохраним результат в best_result_tree.

In [38]:
best_model_tree = None
best_result_tree = 0
best_depth = 0
best_sample = 0

for depth in range(1, 6): # глубина дерева от 1 до 6
    
    for sample in range(10, 100, 10): # минимальное количество выборок для разделения вн. узла 
        
        model = DecisionTreeClassifier(random_state=5, 
                                       max_depth=depth, 
                                       min_samples_split = sample) # создание модели

        model.fit(features_train, target_train) # обучение на обучающей выборке

        result = model.score(features_valid, target_valid) # оценка точности предсказаний на вал. выборке 
        
        if result > best_result_tree: # сохранение параметров лучшей модели
            best_result_tree = result
            best_model_tree = model
            best_depth = depth
            best_sample = sample
        
print(f'Accuracy лучшей модели дерева решений: {best_result_tree:.2%}')
print('Глубина дерева лучшей модели', best_depth)
print('Мин. кол-во выборок для разделения вн. узла:', best_sample)

Accuracy лучшей модели дерева решений: 78.69%
Глубина дерева лучшей модели 5
Мин. кол-во выборок для разделения вн. узла: 80


**Вывод**
***
Наилучший результат предсказаний на валидационной выборке показала модель дерева решений с гиперпараметрами:  
- random_state=5
- max_depth=5, 
- min_samples_split=80.    

Точность предсказаний на валидационной выборке составила 78,69 %.

### Случайный лес

Найдем гиперпараметры для модели случайного леса, с которыми модель дает наилучший результат accuracy.  
Рассмотрим гиперпараметры - n_estimators (количество деревьев), max_depth (глубина деревьев).  
Сохраним наилучшую модель в best_model_forest.

In [39]:
best_result_forest = 0
best_model_forest = None 
best_estimators_forest = 0
best_depth = 0

for est in range(1, 20): # количество деревьев от 1 до 50 
    
    for depth in range(1, 10): # глубина деревьев

        model = RandomForestClassifier(random_state=5,  
                                           n_estimators=est, 
                                           max_depth=depth) # модель

        model.fit(features_train, target_train) # обучение на тренировочной выборке

        result = model.score(features_valid, target_valid) # точность предсказаний на валидацонной выбоке

        if result > best_result_forest: # лучший результат
            best_model_forest = model
            best_result_forest = result
            best_estimators_forest = est
            best_depth = depth

            
print(f'Accuracy наилучшей модели на валидационной выборке: {best_result_forest:.2%}')    
print('Количество деревьев в наилучшей модели', best_estimators_forest)
print('Глубина деревьев', best_depth)

Accuracy наилучшей модели на валидационной выборке: 81.03%
Количество деревьев в наилучшей модели 15
Глубина деревьев 7


**Вывод**
***
Наилучший результат предсказаний на валидационной выборке показала модель случайного леса с гиперпараметрами:  

- random_state=5
- n_estimators=15
- max_depth=7    
    

Точность предсказаний составила 81,03 %.

###  Логистическая регрессия

Обучим модель логистической регрессии с гиперпараметрами:
- random_state
- solver
- penalty
- C

И сохраним модель с наилучшей точностью предсказаний.

In [40]:
model_reg_1 = LogisticRegression(random_state=5, solver='lbfgs')

model_reg_1.fit(features_train, target_train)

result_reg_1 = model_reg_1.score(features_valid, target_valid)

print(f'Accuracy модели на валидационной выборке: {result_reg_1:.2%}') 

Accuracy модели на валидационной выборке: 71.07%


In [41]:
model_reg_2 = LogisticRegression(penalty='none', random_state=5, solver='lbfgs')

model_reg_2.fit(features_train, target_train)

result_reg_2 = model_reg_2.score(features_valid, target_valid)

print(f'Accuracy модели на валидационной выборке: {result_reg_2:.2%}') 

Accuracy модели на валидационной выборке: 75.58%


In [42]:
model_reg_3 = LogisticRegression(penalty='l1', random_state=5, C=.09, solver='liblinear')

model_reg_3.fit(features_train, target_train)

result_reg_3 = model_reg_3.score(features_valid, target_valid)

print(f'Accuracy модели на валидационной выборке: {result_reg_3:.2%}')

Accuracy модели на валидационной выборке: 75.89%


**Вывод**
***
Максимальный результат точности предсказаний модели логистической регрессии составил 75,89%.   

In [43]:
def df_accuracy(m1_tree, m2_forest, m3_reg):
    data = pd.DataFrame({'accuracy': [m1_tree, 
                                      m2_forest, 
                                      m3_reg]}, index=['Дерево решений', 
                                                       'Случайный лес', 
                                                       'Логистическая регрессия']).sort_values(by='accuracy', 
                                                                                               ascending=False)
    return data

### Вывод 
***
Наилучший результат предсказаний на валидационной выборке показала модель случайного леса - accuracy 81,02% с гиперпараметрами:
- random_state=5
- n_estimators=15
- max_depth=7 


In [44]:
df_accuracy(m1_tree=best_result_tree, m2_forest=best_result_forest, m3_reg=result_reg_3)

,accuracy
Случайный лес,0.810264
Дерево решений,0.786936
Логистическая регрессия,0.758942


## Проверка моделей на тестовой выборке

Проверим точность предсказаний лучшей модели дерева решений/случайного леса/лог.регрессии на тестовой выборке. 

Сохраним результаты предсказаний на тестовой выборке.

In [45]:
tree_test = best_model_tree.score(features_test, target_test) # дерево
forest_test = best_model_forest.score(features_test, target_test) # лес
log_reg_test = model_reg_3.score(features_test, target_test) # лог. регрессия

Выведем на экран время, затраченное каждой моделью на предсказание.

In [46]:
%%time
predict_tree = best_model_tree.predict(features_test)

CPU times: user 1.01 ms, sys: 61 µs, total: 1.07 ms
Wall time: 795 µs


In [47]:
%%time
predict_forest = best_model_forest.predict(features_test)

CPU times: user 2.49 ms, sys: 149 µs, total: 2.64 ms
Wall time: 2.45 ms


In [48]:
%%time
predict_reg = model_reg_3.predict(features_test)

CPU times: user 554 µs, sys: 33 µs, total: 587 µs
Wall time: 544 µs


Выведем на экран  в порядке убывания значения точности accuracy для каждой модели на тестовой выборке.

In [49]:
df_accuracy(m1_tree=tree_test, m2_forest=forest_test, m3_reg=log_reg_test)

,accuracy
Дерево решений,0.791602
Случайный лес,0.790047
Логистическая регрессия,0.740280


**Вывод**
***
Почти одинаковые показатели accurancy на тестовой выборке у модели дерева решений - 0,7916 и случайного леса - 0,79. При этом, самое высокое время выполнения предсказаний у модели случайного леса.  
Точность предсказаний у модели логистической регрессии самая низкая 74% и скорость выполнения так же, самая низкая.

## Проверка модели на адекватность

Создадим модель DummyClassifier, в которой все предсказания будут равны наиболее часто встречающемуся признаку в датасете - 0 - тарифу смарт.   
Оценим долю правильных ответов модели и сравним с полученной точностью моделей дерева решений/случайного леса. 

In [50]:
clf = DummyClassifier(strategy='most_frequent', random_state=0)

clf.fit(features_train, target_train)

result_dummy = clf.score(features_test, target_test)

print(f'Точность оценки модели Dummy на тестовой выборке {result_dummy:.2%}')

Точность оценки модели Dummy на тестовой выборке 68.43%


Доля правильных ответов моделей дерева решений и случайного леса выше на ~10% по сравнению с моделью DummyClassifier.

## Вывод

Целью нашего исследования было подобрать модель, рекомендующую пользователям наиболее подходящий для них тариф - смарт или ультра.   
Для этого, в рамках решения задачи классификации было проведено обучение трех моделей:
- дерева решений - DecisionTreeClassifier,
- случайного леса - RandomForestClassifier,
- логистической регрессии - LogisticRegression.  

При этом, исходный датасет был разделен на три выборки: обучающую, валидационную и тестовую (60%, 20%, 20% соответветственно). 

При обучении на обучающей выборке и тестировании моделей на валидационной выборке наилучший результат показала модель случайного леса с гиперпараметрами random_state=5, n_estimators=15, max_depth=7 - точность предсказаний составила 81%.  
Второй по точности предсказаний оказалась модель дерева решений с гиперпараметрами random_state=5, max_depth=5, min_samples_split=80 - точность предсказаний 78,7%.  
Логистическая регрессия показала наименьшую точность предсказаний из всех обученных моделей - 75,9%.   

При тестировании обученных моделей на тестовой выборке результат точности предсказаний оказался практически одинаковым для моделей случайного леса и дерева решений - точность составила 79% и 79,1% соответственно. При этом скорость выполнения предсказаний у модели случайного леса выше, чем у дерева решений. Модель логистической регрессии показала худший результат точности предсказаний на тестовой выборке из всех представленных моделей - 74%, при самой высокой скорости работы.

Так же, стоит отметить, что обученные модели показали более высокую точность предсказаний, по сравнению с моделью DummyClassifier, в которой все предсказания - это самый часты целевой признак - тариф смарт(0). Точность моделей дерева решений и случайного леса оказалась выше на 10%.

**Итак, по результатам исследования и исходя из оценки точности и скорости выполнения предсказаний рекомендуется модель дерева решений с гипермараметрами: random_state=5, max_depth=5, min_samples_split=80 и точностью предсказаний 79,1.%**